# Calculate the terms of the MSE budget

This notebook follows the jas_mean notebook. It calculates each term of the MSE budget. Each term is vertically averaged after calculation
- A. Aiyyer, Aug 2023
- A. Thornton, Aug 2023

In [10]:
import numpy as np
import xarray as xr
import pandas as pd
from datetime import date
from numpy import absolute, exp, log

# Any import of metpy will activate the accessors
from metpy.units import units

#from metpy.calc import dewpoint_from_relative_humidity
from metpy.calc import first_derivative, geospatial_gradient, advection

import metpy.constants as constants

### Path to data
The '3d' directory contains everything we nee to calculate each advection term of the MSE budget. The 2d term folder can just be directly applied to the filter. 

In [11]:
# regridded era5 4x daily data for variables

path_full = '/glade/scratch/athornton/era5_processed_data/3d/' 

### Select subset of dates

In [12]:
year_start  = 1998
month_start = 3
day_start   = 1

year_end  = 2022
month_end = 12
day_end   = 31


date_series = [pd.date_range(date(i,month_start,day_start),date(i,month_end,day_end), freq ='D') for i in range(year_start,year_end+1)]
# date_series is a list of lists. Lets unpack it now

dates_list = [element for sublist in date_series for element in sublist]
print(dates_list[0].strftime("%Y%m%d"))
print(dates_list[-1].strftime("%Y%m%d"))

19980301
19980301


### Mass-weighted vertical integral
Same integral as the vert_integrate notebook

mass weighted vertical integral of a quantity

\begin{align}
[A] = \frac{1}{g}\int_{ps}^{pt} A dp
\end{align}

Which in practice is calculated as:

\begin{align}
[A] = \frac{1}{g} \sum_i \frac{(A[i+1] + A[i])}{2} (p[i]-p[i+1])
\end{align}

Assuming pressure levels go from top (lowest, eg 100 hPa) to bottom (highest, eg. 1000 hPa)

In [13]:
def mass_weighted_vert_integral(data):
    # data is expected to be on pressure levels
    levels = (data.level*units(data.level.units)).metpy.convert_units('Pa')
    deltaP = (levels - levels.shift(level=1)).metpy.convert_units('Pa')
    vert_int_data = ((data.shift(level=1)+data)*.5*deltaP).sum(dim='level') / constants.earth_gravity   
    return  vert_int_data

In [14]:
def print_minmax(var_str, data):
    print( var_str , ' min, max = ', data.min().values, data.max().values, data.metpy.units )
    return

In [24]:
# read the basic state MSE
varNam = 'mse'
variab = 'MSE'

# path to averages
path_bar = '/glade/scratch/athornton/era5_processed_data/jas_means/'+varNam+'_jas_1998_2022.nc'
ds = xr.open_dataset(path_bar)

# the basic state MSE [calculated using mse.ipynb]
hbar = ds[variab].metpy.convert_units('joule/kilogram')
print_minmax('hbar', hbar)

# read the basic state Zonal Wind
varNam = 'u'
variab = 'U'
# path to averages
path_bar = '/glade/scratch/athornton/era5_processed_data/jas_means/'+varNam+'_jas_1998_2022.nc'
ds = xr.open_dataset(path_bar)
# Extract variables
ubar = ds[variab]*units['meter / second']
print_minmax('ubar', ubar)


# read the basic state meridional Wind
varNam = 'v'
variab = 'V'
# path to averages
path_bar = '/glade/scratch/athornton/era5_processed_data/jas_means/'+varNam+'_jas_1998_2022.nc'
ds = xr.open_dataset(path_bar)
# Extract variables
vbar = ds[variab]*units['meter / second']
print_minmax('vbar', vbar)


# read the basic state meridional Wind
varNam = 'w'
variab = 'W'
# path to averages
path_bar = '/glade/scratch/athornton/era5_processed_data/jas_means/'+varNam+'_jas_1998_2022.nc'
ds = xr.open_dataset(path_bar)
# Extract variables
wbar = ds[variab]*units['pascal / second']
print_minmax('wbar', wbar)


hbar  min, max =  234208.42 372106.22 joule / kilogram
ubar  min, max =  -29.158838 22.06894 meter / second
vbar  min, max =  -11.439221 15.893557 meter / second
wbar  min, max =  -0.72537154 1.002546 pascal / second


### Advection terms to calculate:

\begin{align}
U:
-\overline u \frac{\partial h'}{\partial x} 
-\ u' \frac{\partial \overline h}{\partial x}
-\ u' \frac{\partial h'}{\partial x}
\end{align}
\begin{align}
V:
-\overline v \frac{\partial h'}{\partial y}
-\ v' \frac{\partial \overline h}{\partial y}
-\ v' \frac{\partial h'}{\partial y}
\end{align}
\begin{align}
\omega:
-\overline \omega \frac{\partial h'}{\partial p}
-\omega' \frac{\partial \overline h}{\partial p}
-\omega' \frac{\partial h'}{\partial p}
\end{align}

In [7]:
for a_date in dates_list:
    print (a_date)
    # Term 1
    
    # get the 4x daily mse
    path_data = path_full + 'mse_' + a_date.strftime("%Y%m%d") + '.nc'
    ds = xr.open_dataset(path_data)
    h = ds.MSE.metpy.convert_units('joule/kilogram')
    
    # get the 4x daily zonal wind
    path_data = path_full + 'u_' + a_date.strftime("%Y%m%d") + '.nc'
    ds = xr.open_dataset(path_data)
    u = ds.U.metpy.quantify()
    
    
    # get the 4x daily zonal wind
    path_data = path_full + 'v_' + a_date.strftime("%Y%m%d") + '.nc'
    ds = xr.open_dataset(path_data)
    v = ds.V.metpy.quantify()
    
    
    # calculate the anomaly fields
    hp = h - hbar
    print_minmax('mse anomaly', hp)
   

    # calculate the anomaly fields
    up = u - ubar
    print_minmax('u anomaly', up)
   
    vp = v - vbar
    print_minmax('v anomaly', vp)
 
    # calculate the gradient of hp
    hp_dx, hp_dy = geospatial_gradient(hp)
    # Create  new xarray DataArrays with the calculated gradients
    hp_dx = xr.DataArray(hp_dx, coords=hp.coords, dims=hp.dims)
    hp_dy = xr.DataArray(hp_dy, coords=hp.coords, dims=hp.dims)

    # calculate the gradient of the mean MSE (hbar)
    hbar_dx, hbar_dy = geospatial_gradient(hbar)
    # Create new xarray DataArrays with the calculated gradients
    hbar_dx = xr.DataArray(hbar_dx, coords=hbar.coords, dims=hbar.dims)
    hbar_dy = xr.DataArray(hbar_dy, coords=hbar.coords, dims=hbar.dims)
  
    
    #----------------------------------------------------------------------------------------
    # terms = ubar dh'/dx  & vbar dh'/dy    
    # vertically integrated 
    path = '/glade/scratch/athornton/era5_processed_data/budget_terms/unfiltered_terms/'
    
    ubar_hp_dx_vint = mass_weighted_vert_integral(ubar*hp_dx).metpy.convert_units('watt/meter**2')
    ubar_hp_dx_vint.name = 'ubar_hp_dx_vint'
    file_out = path + 'ubar_hp_dx_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'
    ubar_hp_dx_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')
    
    vbar_hp_dy_vint = mass_weighted_vert_integral(vbar*hp_dy).metpy.convert_units('watt/meter**2')
    vbar_hp_dy_vint.name = 'vbar_hp_dy_vint'    
    file_out = path + 'vbar_hp_dy_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'
    vbar_hp_dy_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')
    
    print_minmax('ubar_hp_dx_vint', ubar_hp_dx_vint)

    
    #----------------------------------------------------------------------------------------
    # terms = u' d/dx (hbar) &    v' d/dy (hbar)
    

    up_hbar_dx_vint = mass_weighted_vert_integral(up*hbar_dx).metpy.convert_units('watt/meter**2')
    up_hbar_dx_vint.name = 'up_hbar_dx_vint'    
    file_out = path + 'up_hbar_dx_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    up_hbar_dx_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')

    vp_hbar_dy_vint = mass_weighted_vert_integral(vp*hbar_dy).metpy.convert_units('watt/meter**2')
    vp_hbar_dy_vint.name = 'vp_hbar_dy_vint'    
    file_out = path + 'vp_hbar_dy_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    vp_hbar_dy_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')
   
    print_minmax('up_hbar_dx_vint',up_hbar_dx_vint)

    
    #----------------------------------------------------------------------------------------
    # terms = u' dh'/dx    & v' dh'/dy    
    up_hp_dx_vint = mass_weighted_vert_integral(up*hp_dx).metpy.convert_units('watt/meter**2')
    up_hp_dx_vint.name = 'up_hp_dx_vint'    
    file_out = path + 'up_hp_dx_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    up_hp_dx_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')
    
    vp_hp_dy_vint = mass_weighted_vert_integral(vp*hp_dy).metpy.convert_units('watt/meter**2')
    vp_hp_dy_vint.name = 'vp_hp_dy_vint'    
    file_out = path + 'vp_hp_dy_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    vp_hp_dy_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')
    
    print_minmax('up_hp_dx_vint',up_hp_dx_vint)
  
    
    # now lets do the vertical advection terms
    
    # get the 4x daily vertical wind
    path_data = path_full + 'w_' + a_date.strftime("%Y%m%d") + '.nc'
    ds = xr.open_dataset(path_data)
    w = ds.W.metpy.quantify()
    print_minmax('w',w)

    wp = w - wbar
    print_minmax('wp',wp)
 
    wp_hp_dp_vint = mass_weighted_vert_integral( advection(hp, w=wp)).metpy.convert_units('watt/meter**2')
    wp_hp_dp_vint.name = 'wp_hp_dp_vint'    
    file_out = path + 'wp_hp_dp_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    wp_hp_dp_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')
    print_minmax('wp_hp_dp_vint',wp_hp_dp_vint)
  
    wbar_hp_dp_vint = mass_weighted_vert_integral( advection(hp, w=wbar)).metpy.convert_units('watt/meter**2')
    wbar_hp_dp_vint.name = 'wbar_hp_dp_vint'    
    file_out = path + 'wbar_hp_dp_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    wbar_hp_dp_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')    
    print_minmax('wbar_hp_dp_vint',wbar_hp_dp_vint)
  
    wp_hbar_dp_vint = mass_weighted_vert_integral( advection(hbar, w=wp)).metpy.convert_units('watt/meter**2')
    wp_hbar_dp_vint.name = 'wp_hbar_dp_vint'    
    file_out = path + 'wp_hbar_dp_vint' + '_unfiltered_' + a_date.strftime("%Y%m%d") + '.nc'    
    wp_hbar_dp_vint.metpy.dequantify().to_netcdf(path=file_out, format='NETCDF4', mode='w')    
    print_minmax('wp_hbar_dp_vint',wp_hbar_dp_vint)
  


1998-03-01 00:00:00
mse anomaly  min, max =  -53300.28 34729.594 joule / kilogram
u anomaly  min, max =  -33.853237 80.5354 meter / second
v anomaly  min, max =  -51.4111 44.526592 meter / second
ubar_hp_dx_vint  min, max =  -1911.63509890789 2942.2751320262623 watt / meter ** 2
up_hbar_dx_vint  min, max =  -2934.6129709737966 1272.8408606025278 watt / meter ** 2
up_hp_dx_vint  min, max =  -4084.813304181121 4582.618162287855 watt / meter ** 2
w  min, max =  -9.337384 3.3871677 pascal / second
wp  min, max =  -9.360845 3.3400865 pascal / second
wp_hp_dp_vint  min, max =  -2414.3140021632616 15186.33681649854 watt / meter ** 2
wbar_hp_dp_vint  min, max =  -1165.3962314506089 916.8306443527352 watt / meter ** 2
wp_hbar_dp_vint  min, max =  -18638.77668490677 3048.5384245176106 watt / meter ** 2
1998-03-02 00:00:00
mse anomaly  min, max =  -54117.312 34216.406 joule / kilogram
u anomaly  min, max =  -37.48516 82.5347 meter / second
v anomaly  min, max =  -46.67174 48.55344 meter / second
